# Assignment 3B: Implementing recurrent neural networks in PyTorch


In [1]:
# Standard Library Imports

# Other Libraty Imports
import numpy as np
import torch

# Local Imports
from data_rnn import load_imdb

# Constants


## Part 1 - Classification: data loading

In [ ]:
# ???

## Part 2: Classification, baseline model
### Question 1

In [ ]:
# Create model

### Question 2

In [ ]:
# code

## Part 3: Writing your own Elman RNN
### Question 3

In [ ]:
# code

## Part 3: Using Torch's RNNs
### Question 4

In [ ]:
# code

## Part 4: Autoregressive models
### Question 5

In [2]:
from data_rnn import load_ndfa
from data_rnn import load_brackets
import torch
import torch.nn as nn
import torch.nn.functional as F

# data
x_train_ndfa, (i2w_ndfa, w2i_ndfa) = load_ndfa(n=150_000)
x_train_brack, (i2w_brack, w2i_brack) = load_brackets(n=150_000)

# code

In [3]:

# Create batch separation lines, Append start and end

print(''.join([i2w_ndfa[i] for i in  x_train_ndfa[0]]) )

x_train, (i2w, w2i) = load_brackets(n=150_000)

batches = []
batch_positions = [0]
# max_tokens = 50000
max_batch_length = 64
# token_multiplier = 0
token_counter = 0
y_train = []
for i in range(len(x_train)):
    x_train[i].insert(0,w2i[".start"])
    x_train[i].append(w2i[".end"])
    y_train.append(x_train[i].copy())
    y_train[i][1] = w2i[".unk"]

    if token_counter >= max_batch_length:
        token_counter = 0
        batch_positions.append(i)
        # token_multiplier += 1
    batches.append(len(batch_positions))
    # token_counter += len(x_train[i])
    token_counter += 1
token_sizes = {}
for i in x_train:
    if token_sizes.get(len(i)):
        token_sizes[len(i)] += 1
    else:
        token_sizes[len(i)] = 1


ss


In [4]:
#Separate batches into tensors
x_batches = []
y_batches = []
for i in range(len(batch_positions)-1):
    batch = x_train[batch_positions[i]:batch_positions[i+1]].copy()
    y_batch = y_train[batch_positions[i]:batch_positions[i+1]].copy()
    tok_len = len(batch[-1])
    for j in range(len(x_train[batch_positions[i]:batch_positions[i+1]])):
        if len(batch[j]) < tok_len:
            _temp_list = [w2i[".pad"]]*(tok_len-len(batch[j]))
            batch[j].extend(_temp_list)
            y_batch[j].extend(_temp_list)
    x_batches.append(torch.tensor(batch))
    y_batches.append(torch.tensor(batch))

In [5]:
class RecurrentNN(nn.Module):

    def __init__(self, emb_dim = 32, h_size = 16, lay_num = 1):
        super(RecurrentNN, self).__init__()
        self.embed_num = len(w2i)
        self.emb_dim = emb_dim
        self.h_size = h_size
        self.lay_num = lay_num
        self.embed = nn.Embedding(num_embeddings=self.embed_num, embedding_dim=self.emb_dim)
        self.lstm = nn.LSTM(input_size=self.emb_dim, hidden_size=self.h_size, num_layers=self.lay_num, batch_first=True)
        self.lin = nn.Linear(self.h_size, self.embed_num)


    def forward(self, x):
        x = self.embed(x)
        x, states = self.lstm(x)
        x = self.lin(x)
        return x



In [7]:
# Create RNN object

net = RecurrentNN(lay_num=1)
print(net)

RecurrentNN(
  (embed): Embedding(6, 32)
  (lstm): LSTM(32, 16, batch_first=True)
  (lin): Linear(in_features=16, out_features=6, bias=True)
)


In [6]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


NameError: name 'net' is not defined

In [64]:
import random
random.seed(42)
for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    random.shuffle(x_batches)
    for i, data in enumerate(x_batches, 0):
        # zero the parameter gradients
        optimizer.zero_grad()
        zeros = torch.zeros(64,1)
        target = data.clone()
        target = target.T[1:]
        target = target.T
        target = torch.cat((target, zeros.long()), 1)
        outputs = net(data)
        outputs = outputs.reshape(outputs.shape[0],outputs.shape[2],outputs.shape[1])
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 1.266
[2,  2000] loss: 0.833
[3,  2000] loss: 0.693
[4,  2000] loss: 0.593
[5,  2000] loss: 0.528
Finished Training


In [39]:
max_sequence_length = 160
seq = [[w2i['.start'], w2i['('], w2i['('], w2i[')']]]
seq_len = len(seq[0])
while seq[0][-1] != w2i['.end'] and seq_len <= max_sequence_length:
    seq_tens = torch.tensor(seq)
    seq_tens.shape
    outputs = net(seq_tens)
    _output = []
    dst = outputs[0,-1,: ].detach().numpy()
    prob_dist = np.exp(dst)/sum(np.exp(dst))
    seq_len += 1
    vals_w2i = list(w2i.values())
    # seq[0].append(int(np.random.choice(a=vals_w2i, p=prob_dist)))
    seq[0].append(np.random.choice(vals_w2i,p=prob_dist))
print(seq)

[[1, 5, 5, 4, 4, 2]]


In [40]:
print(''.join([i2w[i] for i in  seq[0]]) )


.start(()).end


In [73]:
PATH = './dyck_model_ep5_seed42_overpredicting_shuffle_per_epoch.pth'
torch.save(net.state_dict(), PATH)

### Question 6


In [74]:
# Sampling definition
import torch.distributions as dist
def sample(lnprobs, temperature=1.0):
    """
    Sample an element from a categorical distribution
    :param lnprobs: Outcome logits
    :param temperature: Sampling temperature. 1.0 follows the given
    distribution, 0.0 returns the maximum probability element.
    :return: The index of the sampled element.
    """
    if temperature == 0.0:
        return lnprobs.argmax()
    p = F.softmax(lnprobs / temperature, dim=0)
    cd = dist.Categorical(p)
    return cd.sample()

In [76]:
# Code

net = RecurrentNN(lay_num=1)
print(net)

import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

max_sequence_length = 160

import random
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    random.seed(42)
    random.shuffle(x_batches)
    for i, data in enumerate(x_batches, 0):
        # zero the parameter gradients
        optimizer.zero_grad()
        zeros = torch.zeros(64,1)
        target = data.clone()
        target = target.T[1:]
        target = target.T
        target = torch.cat((target, zeros.long()), 1)
        # forward + backward + optimize
        outputs = net(data)
        outputs = outputs.reshape(outputs.shape[0],outputs.shape[2],outputs.shape[1])
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
    print(f'Finished Epoch {epoch+1}')
    for seeds in range(10):
        seq = [[w2i['.start'], w2i['('], w2i['('], w2i[')']]]
        seq_len = len(seq[0])
        while seq[0][-1] != w2i['.end'] and seq_len <= max_sequence_length:
            seq_tens = torch.tensor(seq)
            outputs = net(seq_tens)
            dst = outputs[0,-1,: ]
            prob_dist = sample(dst)
            seq_len += 1
            vals_w2i = list(w2i.values())
            # seq[0].append(int(np.random.choice(a=vals_w2i, p=prob_dist)))
            seq[0].append(vals_w2i[prob_dist])
        print(''.join([i2w[i] for i in  seq[0]]) )

RecurrentNN(
  (embed): Embedding(6, 32)
  (lstm): LSTM(32, 16, batch_first=True)
  (lin): Linear(in_features=16, out_features=6, bias=True)
)
[1,  2000] loss: 1.264
Finished Epoch 1
.start(().end
.start(().unk).end
.start(())(.start.start.unk()()().end
.start(())().unk.unk).end
.start(()()().pad(.pad.pad.start.unk.unk.pad)(.end
.start(())()().pad)())).pad(.end
.start(()).unk.end
.start(())().unk.end
.start(())(.end
.start(()()().pad)).end
[2,  2000] loss: 0.784
Finished Epoch 2
.start(()()).end
.start(()()).pad.start.unk.pad)))(.end
.start(()()()).end
.start(()).end
.start(()).unk))).unk.end
.start(().unk.pad.unk).start.unk.end
.start(()().pad.pad).end
.start(()).start.unk.end
.start(().end
.start(().end
[3,  2000] loss: 0.612
Finished Epoch 3
.start(()).end
.start(().start()()().pad.end
.start(()).end
.start(()))).pad.end
.start(()().unk).end
.start(()()).pad.pad.pad)).end
.start(()).end
.start(()).unk.end
.start(().end
.start(()()).end
[4,  2000] loss: 0.544
Finished Epoch 4
.start(

In [77]:
PATH = './q6_dyck_model_ep10_seed42_70_percent.pth'
torch.save(net.state_dict(), PATH)

### Question 7

In [9]:
# code

net = RecurrentNN(lay_num=1)

import random
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

criterion = nn.CrossEntropyLoss(reduction="sum")
optimizer = optim.SGD(net.parameters(), lr=0.0001, momentum=0.9)
writer = SummaryWriter()
for epoch in range(50):  # loop over the dataset multiple times

    running_loss = 0.0
    random.seed(42)
    random.shuffle(x_batches)
    total_norm = 0
    for i, data in enumerate(x_batches, 0):
        # zero the parameter gradients
        optimizer.zero_grad()
        zeros = torch.zeros(64,1)
        target = data.clone()
        target = target.T[1:]
        target = target.T
        target = torch.cat((target, zeros.long()), 1)
        # forward + backward + optimize
        outputs = net(data)
        outputs = outputs.reshape(outputs.shape[0],outputs.shape[2],outputs.shape[1])
        loss = criterion(outputs, target)/len(i2w)
        loss.backward()
        nn.utils.clip_grad_norm_(net.parameters(), max_norm=2.0, norm_type=2)
        optimizer.step()
        # print statistics
        for p in net.parameters():
            param_norm = p.grad.data.norm(2)
            total_norm += param_norm.item() ** 2
        total_norm = total_norm ** (1. / 2)
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
    writer.add_scalar('gd/norm', total_norm/len(x_batches), epoch)
    writer.add_scalar('losstrain', running_loss/len(x_batches), epoch)
    print(f'Finished Epoch {epoch+1}')
writer.close()

RecurrentNN(
  (embed): Embedding(6, 32)
  (lstm): LSTM(32, 16, batch_first=True)
  (lin): Linear(in_features=16, out_features=6, bias=True)
)
[1,  2000] loss: 194.769
Finished Epoch 1
[2,  2000] loss: 177.682
Finished Epoch 2
[3,  2000] loss: 205.706
Finished Epoch 3
[4,  2000] loss: 223.880
Finished Epoch 4
[5,  2000] loss: 209.553
Finished Epoch 5
[6,  2000] loss: 182.446
Finished Epoch 6
[7,  2000] loss: 168.475
Finished Epoch 7
[8,  2000] loss: 158.291
Finished Epoch 8
[9,  2000] loss: 157.141
Finished Epoch 9
[10,  2000] loss: 163.899
Finished Epoch 10
[11,  2000] loss: 153.922
Finished Epoch 11
[12,  2000] loss: 150.888
Finished Epoch 12
[13,  2000] loss: 147.778
Finished Epoch 13
[14,  2000] loss: 147.729
Finished Epoch 14
[15,  2000] loss: 141.561
Finished Epoch 15
[16,  2000] loss: 145.783
Finished Epoch 16
[17,  2000] loss: 137.015
Finished Epoch 17
[18,  2000] loss: 143.662
Finished Epoch 18
[19,  2000] loss: 135.463
Finished Epoch 19
[20,  2000] loss: 134.946
Finished Epoc